In [ ]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, Dataset, Audio, DatasetDict
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
fleurs_dataset = load_dataset("farahabdou/FLEURS-AR-EN-split",trust_remote_code=True)

In [ ]:
fleurs_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'audio', 'arabic', 'english'],
        num_rows: 2228
    })
    validation: Dataset({
        features: ['id', 'audio', 'arabic', 'english'],
        num_rows: 278
    })
    test: Dataset({
        features: ['id', 'audio', 'arabic', 'english'],
        num_rows: 279
    })
})

In [ ]:
df_train = fleurs_dataset["train"].to_pandas().sort_values(by="id")
df_val = fleurs_dataset["validation"].to_pandas().sort_values(by="id")
df_test = fleurs_dataset["test"].to_pandas().sort_values(by="id")

In [ ]:
df_train.shape

(2228, 4)

In [ ]:
df_val.shape

(278, 4)

In [ ]:
df_test.shape

(279, 4)

In [ ]:
df_train = df_train.drop_duplicates(subset=["id"], keep="first").reset_index(drop=True)
df_val = df_val.drop_duplicates(subset=["id"], keep="first").reset_index(drop=True)
df_test = df_test.drop_duplicates(subset=["id"], keep="first").reset_index(drop=True)

In [ ]:
df_train.shape

(1490, 4)

In [ ]:
df_val.shape

(268, 4)

In [ ]:
df_test.shape

(269, 4)

In [ ]:
df_val = df_val[df_val["id"].isin(df_train["id"])]
df_val = df_val.reset_index(drop=True)

In [ ]:
df_val.shape

(166, 4)

In [ ]:
sentences = fleurs_dataset["train"]["arabic"][:5]

# Set the language tokens
tokenizer.src_lang = "ar_Latn"  # Source language: arabic in Latin script
tokenizer.tgt_lang = "eng_Latn"  # Target language: English in Latin script

# Tokenize and set source/target languages
inputs = tokenizer(
    sentences,
    return_tensors="pt",
    padding=True,
    truncation=True
)

# Generate translations
# Instead of using forced_bos_token_id with get_lang_id, provide the target language ID directly
# Get the target language token ID
target_lang_token_id = tokenizer.convert_tokens_to_ids(tokenizer.tgt_lang)

# Pass the target language token ID to forced_bos_token_id
translated = model.generate(**inputs, forced_bos_token_id=target_lang_token_id)
translations = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# Print translations
for i, translation in enumerate(translations):
    print(f"Input: {sentences[i]}")
    print(f"Translation: {translation}\n")

Input: في حالة عدم تواجد أي مكواة، أو إن كنت لا تفضل ارتداء الجوارب المكوية، إذن تستطيع أن تجرب استخدام مجفف الهواء، إذا كان متاحاً.
Translation: If there is no vacuum, or if you do not prefer to wear socks, then you can try using an air dryer, if available.

Input: لم يبدُ لي ذلك منطقياً؛ بالتأكيد لم يكن عادلاً.
Translation: That didn't make sense to me; it certainly wasn't fair.

Input: هذا يجعلُ استخدامَ عدساتِ التّكْبيرِ الرخيصةِ صعبًا في ظروفِ الإضاءةِ المنخفضةِ بدون فلاش.
Translation: This makes it difficult to use cheap magnifying lenses in low light conditions without flashing.

Input: من المتوقع أن أغلب تذاكر الحدث ستبلغ تكلفتها بين 2,500 ين و 130,000 ين أما كلفة العادية النموذجية فستكون حوالي 7,000 ين.
Translation: Most tickets for the event are expected to cost between 2,500 yen and 130,000 yen, while the typical cost will be around 7,000 yen.

Input: لا تُعرف في هذا الوقت التهم التي سيتم توجيهها أو ما الذي أرشد السلطات إلى الصبي ولكن الإجراءات المتعلقة بالأحداث قد بدأت في ا

In [ ]:

# Preprocess dataset
def preprocess_function(dataset):
    model_inputs = tokenizer(
        dataset["arabic"],  # Source text column
        text_target=dataset["english"],  # Target text column
        max_length=128,  # Adjust max length based on your needs
        truncation=True,  # Truncate inputs that exceed the max length
        padding="max_length"  # Pad inputs to the max length
    )
    return model_inputs

tokenized_datasets = fleurs_dataset.map(preprocess_function, batched=True)



Map:   0%|          | 0/2228 [00:00<?, ? examples/s]

Map:   0%|          | 0/278 [00:00<?, ? examples/s]

Map:   0%|          | 0/279 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'audio', 'arabic', 'english', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2228
    })
    validation: Dataset({
        features: ['id', 'audio', 'arabic', 'english', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 278
    })
    test: Dataset({
        features: ['id', 'audio', 'arabic', 'english', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 279
    })
})

In [ ]:
# Fine-tune
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",                # Directory to save checkpoints
    evaluation_strategy="epoch",          # Evaluate at the end of each epoch
    learning_rate=5e-5,                    # Learning rate
    per_device_train_batch_size=2,        # Reduce batch size to 2
    per_device_eval_batch_size=2,         # Reduce evaluation batch size to 2
    gradient_accumulation_steps=8,        # Simulate larger batch size (effective batch size = 2 * 8 = 16)
    num_train_epochs=10,                  # Reduce epochs for quicker testing
    save_total_limit=2,                   # Limit saved checkpoints to save memory
    predict_with_generate=True,           # Generate predictions for validation
    logging_dir="./logs",                 # Log directory
    logging_steps=100,                    # Log every 100 steps (reduce logging overhead)
    push_to_hub=True,                     # Push model to Hugging Face Hub
    hub_model_id="farahabdou/nllb-AR-ENG", # Model ID on Hugging Face Hub
    fp16=True,                            # Enable mixed precision training
    save_strategy="epoch",                # Save checkpoint at the end of each epoch
    save_steps=None,                      # Avoid saving too frequently
    max_steps=1000,                       # Optionally limit the total number of training steps
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-30-bd5cc11ee9d8>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
import torch
torch.cuda.empty_cache()

## Training

In [ ]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.341900,0.151657
2,0.276400,0.153033
3,0.158100,0.152472
4,0.123200,0.156430
5,0.091400,0.159405
6,0.072000,0.161156
7,0.063800,0.161744


TrainOutput(global_step=1000, training_loss=0.1524219045639038, metrics={'train_runtime': 4267.4556, 'train_samples_per_second': 3.749, 'train_steps_per_second': 0.234, 'total_flos': 4311454605901824.0, 'train_loss': 0.1524219045639038, 'epoch': 7.143626570915619})

In [2]:
!pip install datasets jiwer sacrebleu

In [1]:
!pip install evaluate

## Evaluation

In [1]:
from transformers import pipeline
from datasets import load_dataset
import evaluate
import torch
import sacrebleu

# Load components
asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-small",
    device="cuda" if torch.cuda.is_available() else "cpu",
    generate_kwargs={"task": "transcribe", "language": "ar"}
)

mt_pipeline = pipeline(
    "translation",
    model="facebook/nllb-200-distilled-600M",
    src_lang="ara_Arab",  # NLLB language code for Arabic
    tgt_lang="eng_Latn",  # NLLB language code for English
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# Load dataset
dataset = load_dataset("farahabdou/FLEURS-AR-EN-split", split="test")

def cascaded_translation(example):
    # Stage 1: ASR
    audio_array = example["audio"]["array"]
    sample_rate = example["audio"]["sampling_rate"]

    asr_result = asr_pipeline(
        {"raw": audio_array, "sampling_rate": sample_rate},
        chunk_length_s=30,
        stride_length_s=5
    )["text"]

    # Stage 2: Translation
    try:
        mt_result = mt_pipeline(
            asr_result,
            max_length=400,
            clean_up_tokenization_spaces=True
        )[0]["translation_text"]
    except Exception as e:
        print(f"Translation error: {str(e)}")
        mt_result = ""

    return {"asr_prediction": asr_result, "mt_prediction": mt_result}

# Process dataset
results = dataset.map(cascaded_translation, batched=False, batch_size=1)

# Calculate metrics
def evaluate_cascade(results):
    # ASR Evaluation (WER)
    wer = evaluate.load("wer")
    asr_wer = wer.compute(
        references=results["arabic"],
        predictions=results["asr_prediction"]
    )

    # Translation Evaluation (BLEU/ChrF)
    bleu = sacrebleu.corpus_bleu(results["mt_prediction"], [results["english"]])
    chrf = sacrebleu.corpus_chrf(results["mt_prediction"], [results["english"]])

    return {
        "asr_wer": asr_wer * 100,
        "translation_bleu": bleu.score,
        "translation_chrf": chrf.score
    }

metrics = evaluate_cascade(results)

# Print results
print(f"ASR Word Error Rate: {metrics['asr_wer']:.2f}%")
print(f"Translation BLEU: {metrics['translation_bleu']:.2f}")
print(f"Translation ChrF++: {metrics['translation_chrf']:.2f}")

# Show samples
print("\nSample Translations:")
for i in range(3):
    print(f"Audio: {dataset[i]['path']}")
    print(f"ASR Arabic: {results[i]['asr_prediction']}")
    print(f"Translated English: {results[i]['mt_prediction']}")
    print(f"Reference English: {results[i]['english']}\n")

Device set to use cuda
Device set to use cuda


Map:   0%|          | 0/279 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


ASR Word Error Rate: 43.49%
Translation BLEU: 22.58
Translation ChrF++: 50.45

Sample Translations:


KeyError: 'path'

In [4]:
!pip install unbabel-comet

In [2]:
# Add COMET evaluation
import evaluate # Import the evaluate module

comet = evaluate.load('comet')

def evaluate_cascade(results):
    # Existing metrics
    wer = evaluate.load("wer")
    asr_wer = wer.compute(
        references=results["arabic"],
        predictions=results["asr_prediction"]
    )

    bleu = sacrebleu.corpus_bleu(results["mt_prediction"], [results["english"]])
    chrf = sacrebleu.corpus_chrf(results["mt_prediction"], [results["english"]])

    # Add COMET score
    comet_score = comet.compute(
        predictions=results["mt_prediction"],
        references=results["english"],
        sources=results["arabic"]
    )

    return {
        "asr_wer": asr_wer * 100,
        "translation_bleu": bleu.score,
        "translation_chrf": chrf.score,
        "translation_comet": comet_score["mean_score"]
    }

metrics = evaluate_cascade(results)

# Print updated results including COMET
print(f"ASR Word Error Rate: {metrics['asr_wer']:.2f}%")

print(f"Translation BLEU: {metrics['translation_bleu']:.2f}")
print(f"Translation ChrF++: {metrics['translation_chrf']:.2f}")
print(f"Translation COMET: {metrics['translation_comet']:.4f}")


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/f49d328952c3470eff6bb6f545d62bfdb6e66304/checkpoints/model.ckpt`
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


ASR Word Error Rate: 43.49%
Translation BLEU: 22.58
Translation ChrF++: 50.45
Translation COMET: 0.7195
